# Librerias

In [1]:
!pip install kafka-python
!pip install confluent-kafka

In [2]:
import pandas as pd 
import json
import random 
import time
import hashlib


from kafka import KafkaProducer
from confluent_kafka import Producer
import socket
import json
import os

from kafka_auth import conf



In [3]:
ciudades = ['Capital_Federal','Cordoba','Rosario','Resistencia','Posadas']
pago_en_linea = ['Tarjeta_de_credito','Mercado_Pago']
pago_en_tienda = ['Efectivo','Tarjeta_de_credito','Mercado_Pago']
origen = ['Facebook','Instagram','Organic','Twitter','Influencer_1','Influencer_2','Influencer_3','Influencer_4']
estado_compra = ['COMPLETADO','PAGO_FALLIDO','FALLO_RESPUESTA_API','FONDOS_INSUFICIENTES','COMPLETADO','COMPLETADO','COMPLETADO','COMPLETADO','COMPLETADO','COMPLETADO','FALLO_RESPUESTA_API','FONDOS_INSUFICIENTES','ERROR_USUARIO','FRAUDE','COMPLETADO','COMPLETADO','COMPLETADO']

In [4]:
cap_coords = [
    (-34.588571, -58.411379),
    (-34.603738, -58.375216),
    (-34.606232, -58.409714)]
cor_coords = [
    (-31.418623, -64.190728),
    (-31.436559, -64.183577),
    (-31.405747, -64.195188)
]
ros_coords = [
    (-32.956032, -60.646908),
    (-32.947581, -60.641690),
    (-32.957750, -60.641780)
]
res_coords = [
    (-27.451214, -58.986389),
    (-27.445978, -58.978493),
    (-27.446658, -58.985378)
]
pos_coords = [
    (-27.369140, -55.907111),
    (-27.362986, -55.907959),
    (-27.366249, -55.906467)
]

In [5]:
def obtener_metodo_pago(origen, estado_compra, pago_en_linea, pago_en_tienda):
    
    if origen == 'Orgánico':
        
        pago = random.choice(pago_en_tienda)
        estado = 'COMPLETADO'
        tipo_orden = 'TIENDA'
        
    elif origen != 'Orgánico':
        
        pago = random.choice(pago_en_linea)
        estado = random.choice(estado_compra)
        tipo_orden = 'EN LÍNEA'
    
    return pago, estado, tipo_orden


def obtener_coordenadas(ciudad):
    coords = None
    
    if ciudad == 'Capital_Federal':
        coords = random.choice(cap_coords)
    elif ciudad == 'Cordoba':
        coords = random.choice(cor_coords)
    elif ciudad == 'Rosario':
        coords = random.choice(ros_coords)
    elif ciudad == 'Resistencia':
        coords = random.choice(res_coords)
    elif ciudad == 'Posadas':
        coords = random.choice(pos_coords)
        
    return coords



In [6]:
pago = obtener_metodo_pago(origen, estado_compra, pago_en_linea, pago_en_tienda)
print(pago)

('Tarjeta_de_credito', 'COMPLETADO', 'EN LÍNEA')


In [7]:
pago[0]

'Tarjeta_de_credito'

In [8]:
pago[1]

'COMPLETADO'

In [9]:
random.choice(ciudades)

'Capital_Federal'

In [10]:
prueba = random.choice(ciudades)
print(f' Ciudad  >> {prueba}')
print(f' Coordenadas >> {obtener_coordenadas(prueba)}')

 Ciudad  >> Resistencia
 Coordenadas >> (-27.446658, -58.985378)


In [11]:
df = pd.read_excel('ecomerce_datexland.xlsx')

In [12]:
df

,NOMBRE_PRODUCTO,PRECIO,COMISION,MARCA,CATEGORIA
0,Laptop_Brand1_4GB,135000,0.20,Brand1,LAPTOPS
1,Laptop_Brand1_8GB,255000,0.25,Brand1,LAPTOPS
2,Laptop_Brand1_16GB,350000,0.30,Brand1,LAPTOPS
3,Laptop_Brand1_32GB,480000,0.35,Brand1,GAMING
4,Laptop_Brand2_4GB,115000,0.15,Brand2,LAPTOPS
5,Laptop_Brand2_8GB,185000,0.18,Brand2,LAPTOPS
6,Laptop_Brand2_16GB,389000,0.20,Brand2,LAPTOPS
7,Laptop_Brand2_32GB,699000,0.25,Brand2,GAMING
8,Laptop_Brand3_4GB,185000,0.20,Brand3,LAPTOPS
9,Laptop_Brand3_8GB,319900,0.28,Brand3,LAPTOPS


# Configuracion Cluster Kafka

In [13]:
# conf = ccloud_lib.read_ccloud_config('librdkafka.config')

# # Create Producer instance
# producer_conf = ccloud_lib.pop_schema_registry_params_from_config(conf)
# producer = Producer(producer_conf)

producer = Producer(conf)

In [14]:
registros_entregados = 0
topic = 'topic_Druid'

x = 1

data_purchase = []

while(x > 0):
    
    fecha = pd.to_datetime('today').strftime("%Y-%m-%d %H:%M:%S")
    
    producto = df['NOMBRE_PRODUCTO'][random.randint(0, 14)]
    precio = df[df['NOMBRE_PRODUCTO'] == producto]['PRECIO'].values[0]
    comision = df[df['NOMBRE_PRODUCTO'] == producto]['COMISION'].values[0]
    marca = df[df['NOMBRE_PRODUCTO'] == producto]['MARCA'].values[0]
    categoria = df[df['NOMBRE_PRODUCTO'] == producto]['CATEGORIA'].values[0]
    fuente_temp = random.choice(origen)
    pago = obtener_metodo_pago(fuente_temp, estado_compra, pago_en_linea, pago_en_tienda)
    ciudad = random.choice(ciudades)
    
    compra = {
        'ID_compra': str(hashlib.sha256(f"{x} {producto} {precio} {comision} {fecha} {fuente_temp} {pago[1]}".encode('utf-8')).hexdigest())[:10],
        'Nombre_producto': producto,
        'Precio': str(precio),
        'Comision': str(comision),
        'Ingresos': str(round(precio * comision, 2)),
        'Metodo_pago': pago[0],
        'Estado': pago[1],
        'Tipo_orden': pago[2],
        'Ciudad': ciudad,
        'Ubicacion': str(obtener_coordenadas(ciudad)),
        'Latitud': str(obtener_coordenadas(ciudad)[0]),
        'Longitud': str(obtener_coordenadas(ciudad)[1]),
        'Fuente': fuente_temp,
        'Marca': marca,
        'Categoria': categoria,
        'Creado_en': fecha
    }
    
    # data_purchase.append(pd.DataFrame(compra))

    record_key = "Simulador_Compras"
    record_value =  json.dumps(compra).encode('utf-8')
    print("Producing record: {}\t{}".format(record_key, record_value))
    producer.produce(topic, key=record_key, value=record_value)
    producer.poll(0)
    registros_entregados += 1
    x += 1
    
    time.sleep(random.choice([1,1.5]))

productor.flush()

print('\n')
print('\n')
print("¡Se produjeron {} mensajes en el tema {}!".format(registros_entregados, topic))
print('\n')
print('\n')

Producing record: Simulador_Compras	b'{"ID_compra": "f5616fc423", "Nombre_producto": "Laptop_Brand3_16GB", "Precio": "465900", "Comision": "0.3", "Ingresos": "139770.0", "Metodo_pago": "Tarjeta_de_credito", "Estado": "COMPLETADO", "Tipo_orden": "EN L\\u00cdNEA", "Ciudad": "Cordoba", "Ubicacion": "(-31.405747, -64.195188)", "Latitud": "-31.418623", "Longitud": "-64.190728", "Fuente": "Facebook", "Marca": "Brand3", "Categoria": "LAPTOPS", "Creado_en": "2023-05-14 01:05:10"}'
Producing record: Simulador_Compras	b'{"ID_compra": "40f2a88309", "Nombre_producto": "Laptop_Brand2_8GB", "Precio": "185000", "Comision": "0.18", "Ingresos": "33300.0", "Metodo_pago": "Tarjeta_de_credito", "Estado": "COMPLETADO", "Tipo_orden": "EN L\\u00cdNEA", "Ciudad": "Capital_Federal", "Ubicacion": "(-34.588571, -58.411379)", "Latitud": "-34.603738", "Longitud": "-58.409714", "Fuente": "Influencer_1", "Marca": "Brand2", "Categoria": "LAPTOPS", "Creado_en": "2023-05-14 01:05:11"}'
Producing record: Simulador_Compr

KeyboardInterrupt: 